In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
books = pd.read_csv('/content/drive/MyDrive/Projects/Book/archive/Books.csv')
books.head()

In [ ]:
users = pd.read_csv('/content/drive/MyDrive/Projects/Book/archive/Users.csv')
users.head()

In [ ]:
ratings = pd.read_csv('/content/drive/MyDrive/Projects/Book/archive/Ratings.csv')
ratings.head()

# **EDA**

In [ ]:
print(books.shape)
print(users.shape)
print(ratings.shape)

In [ ]:
books.isna().sum()

In [ ]:
users.isna().sum()

In [ ]:
ratings.isna().sum()

In [ ]:
ratings.dtypes

In [ ]:
books.dtypes

In [ ]:
books['Image-URL-L'].fillna(books['Image-URL-L'].mode(), inplace=True)

In [ ]:
books['Publisher'].fillna(books['Publisher'].mode(), inplace=True)

In [ ]:
books['Book-Title'].fillna(books['Book-Title'].mode(), inplace=True)

In [ ]:
users['Age'].fillna(users['Age'].mean(), inplace=True)

In [ ]:
ratings['Book-Rating'].fillna(ratings['Book-Rating'].mean(), inplace=True)

In [ ]:
ratings['Book-Rating'].replace('006250746X006250746X006250746X006250746X006250746X006250746X006250746X006250746X006250746X006250746X',0,inplace = True)

In [ ]:
books.isna().sum()

In [ ]:
users.isna().sum()

In [ ]:
books.duplicated().sum()

In [ ]:
books.describe()

# **Correlation matrix**

In [ ]:
# plt.figure(figsize=(10, 8))
# sns.heatmap(users.corr(), annot=True, cmap='coolwarm')
# plt.show()

In [ ]:
# plt.figure(figsize=(10, 8))
# sns.heatmap(ratings .corr(), annot=True, cmap='coolwarm')
# plt.show()

# **Merge two dfs**

In [ ]:
merge_df = ratings.merge(books, on='ISBN')

In [ ]:
merge_df.head()

# **Collaborative based**

In [ ]:
temp = merge_df.groupby('User-ID').count()['Book-Rating'] > 200
rated_users = temp[temp].index

In [ ]:
merge_df['User-ID'].isin(rated_users)
filtered_rating = merge_df[merge_df['User-ID'].isin(rated_users)]

In [ ]:
filtered_rating.shape

In [ ]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50
popular_books = y[y].index

In [ ]:
filtered_rating['Book-Title'].isin(popular_books)
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(popular_books)]

In [ ]:
final_ratings.drop_duplicates

In [ ]:
final_ratings.shape

In [ ]:
fr = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
fr.fillna(0,inplace = True)

In [ ]:
fr

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_score = cosine_similarity(fr)

In [ ]:
fr

In [ ]:
similarity_score[0]

In [ ]:
def recommend(book_name):
  book_index = np.where(fr.index==book_name)[0][0]
  distances = similarity_score[book_index]
  similar_items = sorted(list(enumerate(distances)),key=lambda x:x[1],reverse=True)[1:6]

  for i in similar_items:
    print(fr.index[i[0]])

In [ ]:
recommend('Message in a Bottle')

In [ ]:
recommend('1984')

In [ ]:
import pickle
pickle.dump(final_ratings,open('final_ratings.pkl','wb'))